<h1> Load the required libraries..</h1>

In [1]:
import random
import pandas as pd
from tqdm import tqdm

<H1> Load the Dataset.. </H1>

In [2]:
df = pd.read_csv("Codewars Data.csv")       # Load the csv file..

In [3]:
df.head()

,Team Name,Initial Rting,Language of Code
0,A01,77,C++
1,A02,60,P
2,A03,83,P
3,A04,84,P
4,A05,71,P


In [4]:
len(df)

50

<h1>Flow..</h1>
    <ul>
        <li>Initial rating</li>
        <li>Outcomes</li>
        <li>Adjustments</li>
        <li>LeaderBoard</li>
    </ul>

In [20]:
# Inital Rating is already done.. So we have to do only rest 3 steps repeatdly..

# Create Team class to load all the teams into the torunament..

In [21]:
class Team:
    def __init__(self, name, rating , lang):
        self.name = name
        self.rating = rating
        self.lang = lang
        self.score = 0        # No. of matches won..
        self.opp_ratings = []   # To break tie..
        self.opp_teams = []     # To prevent same opponent..
        self.loose_times = []   # To break tie..
        self.win_times = []     # To break tie..

# Class to load teams, pair them then simulate match and fetch leaderboard..

In [37]:
class SwissPairTournament:
    
    def __init__(self, teams):    # Constructor of the class... I'm Loading the teams here..
        self.teams = teams
        self.pairings = []        # To store the paired teams tuple...
        self.byes = []            # To store teams which cannot find any other team to match..

    def pair_teams(self):

        random.shuffle(self.teams)   # To bring randomness ...
 
        paired = [] 
        self.pairings = []
        self.byes = []

        # logic - for every team in self.teams check whether the team is paired or not... if paired skip that iteration.. 
        # otherwise find another team from self.teams with some constraints..
        # constraints for opponent team - opp should also be not paired..
        # opp should have same language of code as team, opp should not be paired with this team before..
        # Also try to get the rating difference between both the teams as small as poosible..
        
        for team in self.teams:       
            if team not in paired:
                best_opponent = None
                best_diff = 99999999999999

                for opp_team in self.teams:
                    if (team != opp_team) and (opp_team not in paired) and (team.lang == opp_team.lang) and (opp_team not in team.opp_teams) :
                        rating_diff = abs(team.rating - opp_team.rating)
                        if rating_diff < best_diff:
                            best_diff = rating_diff
                            best_opponent = opp_team

                paired.append(team)    
                if best_opponent != None:          # If opponent is found..
                    self.pairings.append((team, best_opponent))
                    paired.append(best_opponent)
                else:                               # If opponent is not found..
                    self.byes.append(team)
                
        return self.pairings, self.byes

                
    def get_noise(self):                # This fucntion generates noise for the current rating..
        return random.randint(-4,4)
    
    def predict_winner(self,team, opp_team):
        
        # Calculate the ratings of both teams (after adding noise)
        team_rating = team.rating + self.get_noise()
        opp_team_rating = opp_team.rating + self.get_noise()

        rating_diff = abs(team_rating - opp_team_rating)
        
        # 1 means team wins.. 0 means opp_team wins
        if rating_diff < 5:
            return random.choice([0,1])
        elif rating_diff <= 10:

            if team.rating > opp_team.rating:
                return random.choice([1]*65 + [0]*35)    # list contains 65 1 and 35 0.. 
            else: 
                return random.choice([0]*65 + [1]*35)
        else:
            if team.rating > opp_team.rating:
                return random.choice([1]*90 + [0]*10)
            else: 
                return random.choice([0]*90 + [1]*10)
            
    def match_teams(self):

        win_history = []

        # Iterate over the pairings...
        for team, opp_team in self.pairings:

            # Match between the team and opp team..
            winner = self.predict_winner(team,opp_team)

            change = 0    # To update ratings after each match..
            match_time = random.randint(5,10)   # Time duration of the match..
            
            if winner:    # team wins..

                team.score += 1  # Increase the score..
                team.win_times.append(match_time)
                opp_team.loose_times.append(match_time)
                
                if team.rating >= opp_team.rating:
                    # Update the ratings by +2 and -2..
                    change = 2
                else:
                    # Update the ratings by +5 and -5..
                    change = 5
                    
            else:          # opp_team wins
                
                opp_team.score += 1  # Increase the score..
                opp_team.win_times.append(match_time)
                team.loose_times.append(match_time)
                
                if team.rating <= opp_team.rating:
                    # Update the ratings by +2 and -2..
                    change = 2
                    
                else:
                    # Update the ratings by +5 and -5..
                    change = 5
                    
            # Add the opponent teams in both teams to avoid future clash (again)..
            team.opp_teams.append(opp_team)
            opp_team.opp_teams.append(team)

            # Add the opponent rating in both teams opp_ratings to use in break tie fucntion..
            team.opp_ratings.append(opp_team.rating)
            opp_team.opp_ratings.append(team.rating)

            # Finally update the ratings of both teams and append the win into the win_history list to track history..
            self.update_ratings(team, opp_team, winner, change)
            win_history.append(winner)

        
        # Iterate over byes teams..
        for team in self.byes:

            # Give a bye to the team
            team.score += 1  # Increase Score..
            team.rating += 2 # Give a bye as defeating a lower rated opp..

            # Add the rating in team opp_ratings to use in break tie fucntion..
            team.opp_ratings.append(team.rating)
        
        return win_history
            
    
    def update_ratings(self, team , opp_team, win_team , change):
        
        if win_team:   # team has won
            team.rating += change
            opp_team.rating -= change
            
        else:   # opp_team has won..
            team.rating -= change
            opp_team.rating += change

    def break_tie(self, teams):
        # print(f"Ties: {len(teams)}")
        
        # Recieve all the teams with same score..
        # Now break the tie on the basis of ratings of the opponents they have faced (better rating means better ranking) and ..
        # Duration of previous matches.. (team wins in lesser time or loses in more time, it should be ranked better)

        # Equation to comapre is .. opp_ratings + x * (loss_time - win_time) (x>0) . Higher the value, higer the ranking..
        # We have to take some value of x according to the importance of time duration in comaprision to ratings..

        tie_scores = []
        for team in teams:

            # Average out the opponent ratings for comparision..
            if len(team.opp_ratings) > 0:
                rating_sum = 0
                for rating in team.opp_ratings:
                    rating_sum += rating
                opp_rating = rating_sum/ len(team.opp_ratings)
            else:           # Avoid zero division error..
                opp_rating = 0

            # Average out winning time and lossing time..
            if len(team.win_times) > 0:
                win_time_sum = 0
                for win_time in team.win_times:
                    win_time_sum += win_time
                avg_win_time = win_time_sum/ len(team.win_times)
            else:           # Avoid zero division error..
                avg_win_time = 0

            # Loosing time..
            if len(team.loose_times) > 0:
                loose_time_sum = 0
                for loose_time in team.loose_times:
                    loose_time_sum += loose_time
                avg_loose_time = loose_time_sum/ len(team.loose_times)
            else:           # Avoid zero division error..
                avg_loose_time = 0

            
            tie_scores.append(opp_rating + 1 * (avg_loose_time - avg_win_time)) 
            # 1 means time is as important as opponents rating.. And this can be changed accordingly to our requirements..

        # Now sort the tie_scores according to their value first by converting them into dictionary to store index values (to retain identity)..
        scores_dict = enumerate(tie_scores)
        scores_dict = sorted(scores_dict, key = lambda x: x[1], reverse = True)  # Sort them in descending order acc to score..

        # Finally make the return list according to the sorted dictionary..
        return_teams = []
        for index , score in scores_dict:
            return_teams.append(teams[index])    # Add teams according to thir highest scores..
    
        return return_teams
    
    def get_leaderboard(self):

        leaderboard = []

        # All teams are in self.teams (list)
        # Sort them according to their ratings and break the tie if it is there..
        
        sorted_teams = sorted(self.teams, key = lambda x: x.score, reverse=True)
        
        team_index = 0        
        while team_index < len(sorted_teams):
            current_score = sorted_teams[team_index].score

            count = 0
            while (team_index + count < len(sorted_teams)) and (sorted_teams[team_index:][count].score == current_score):
                count += 1
                
            if count > 1:
                leaderboard += self.break_tie(sorted_teams[team_index: team_index + count])
            else:
                leaderboard += [sorted_teams[team_index]]
            team_index += count
        return leaderboard

<li> Some helper functions.. </li>

In [38]:
# To convert Team Objects into dataframe...

def to_df(teams):

    return_df = pd.DataFrame(columns= ['Team Name', 'Rating', 'Language of Code', 'Score'])
    
    for team in teams:
        return_df = return_df._append(pd.Series((team.name, team.rating, team.lang, team.score) , index = ['Team Name', 'Rating', 'Language of Code', 'Score']), ignore_index = True)
    
    return_df = pd.DataFrame(return_df.values, columns = return_df.columns, index=range(1, len(return_df) + 1))
    return return_df        

In [39]:
# To convert Team winning history into dataframe format..

def to_windf(teams_pair, win_history):

    return_df = pd.DataFrame(columns= ['Team Name', 'Rating', 'Language of Code', 'Score'])

    for pair_index in range(len(teams_pair)):
        if win_history[pair_index]:
            team = teams_pair[pair_index][0]
        else:
            team = teams_pair[pair_index][1]
            
        return_df = return_df._append(pd.Series((team.name, team.rating, team.lang, team.score) , index = ['Team Name', 'Rating', 'Language of Code', 'Score']), ignore_index = True)
    
    return_df = pd.DataFrame(return_df.values, columns = return_df.columns, index=range(1, len(return_df) + 1))
    return return_df

<h1> Initialise Teams objects and tournament instance..</h1>

<li> Initialise the team objects for every team in the csv file ..</li>

In [40]:
teams = []
for _ , row in df.iterrows():
    teams.append(Team(name = row['Team Name'], rating = row['Initial Rting'], lang = row['Language of Code']))

<li> Make a tournament object to simulate the coding tournament according to the rules... </li>

In [41]:
tournament = SwissPairTournament(teams)

<h1>Main Function to do all the tasks.. </h1>
    <ul>
    <li> Pairing of Teams</li>
    <li> Simulating the Match between Teams</li>
    <li> Update and Display LeaderBoard</li>
    </ul>



In [42]:
# Function will do all the tasks and finally calcualte and present the result...
def eval_result(tournament, rounds, MAX_BYES, MAX_PAIR_ITER = 500):
    
    for round in tqdm(range(rounds)):   # Iter rounds as argument..
        
        # Pair the teams according to their rating..
        # Ensure that there should not be more then MAX_BYES byes.. (according to given data)
        byes = [0]*10
        iter = 0
        while len(byes) > MAX_BYES and iter < MAX_PAIR_ITER:    # Find new pairs untill get such combiantion which have less than MAX_BYES byes..
            pairings , byes  = tournament.pair_teams()      
            iter += 1                                      # Also ensure that iterations does not cross its max value (infinite loop)..
            
        print(f"The pairings for the round number {round+1} are:") # Print the paired teams..
        for pair in pairings:                # Print after converting them to pandas dataframe.. 
            print(to_df(pair))
        print("\n")
        
        print(f"The teams having byes in the round number {round+1} are:") # Print the bye teams..
        print(to_df(byes))
        print("\n")

        
        # Match the teams and print the winners..
        win_history = tournament.match_teams()
        print(f"The winning teams for the round number {round+1} are:") # Print the winning teams..
        print(to_windf(pairings, win_history))   # Convert to dataframe and then print..
        print("\n")

        
        # Update leaderboard and display it..
        leaderboard = tournament.get_leaderboard()
        print(f"The leaderboard for the round number {round+1} is:")
        print(to_df(leaderboard))                # Convert to datafarme and then print the leaderboard..

        print("\n----------------------------------------------------------------\n\n")
    
    print("Final Leaderboard:\n")
    print(to_df(leaderboard))

# Evaluate over 10 rounds..

<li>Show Paired Teams</li>
<li>Show byes Teams </li>
<li>Show winning Teams</li>
<li>Show Updated Leaderboard</li>
<li>Repeat these steps 10 times</li>
<li>Finally show the final leaderboard</li>

In [43]:
eval_result(tournament, 10, 2)

 10%|████████▎                                                                          | 1/10 [00:00<00:01,  5.35it/s]

The pairings for the round number 1 are:
  Team Name Rating Language of Code Score
1       A37     79                P     0
2       A18     79                P     0
  Team Name Rating Language of Code Score
1       A03     83                P     0
2       A04     84                P     0
  Team Name Rating Language of Code Score
1       A45     92                P     0
2       A24     92                P     0
  Team Name Rating Language of Code Score
1       A09     80              C++     0
2       A08     82              C++     0
  Team Name Rating Language of Code Score
1       A48     93                P     0
2       A31     93                P     0
  Team Name Rating Language of Code Score
1       A42     85                P     0
2       A23     86                P     0
  Team Name Rating Language of Code Score
1       A17     89                P     0
2       A41     89                P     0
  Team Name Rating Language of Code Score
1       A01     77              C++

 30%|████████████████████████▉                                                          | 3/10 [00:00<00:01,  6.16it/s]

   Team Name Rating Language of Code Score
1        A03     90                P     2
2        A22    101              C++     2
3        A31     97                P     2
4        A40     81                P     1
5        A36     79                P     2
6        A20     97              C++     2
7        A21     75                P     1
8        A02     67                P     2
9        A19     73                P     2
10       A48     93                P     1
11       A42     85                P     1
12       A09     83              C++     1
13       A47     88              C++     1
14       A12     89                P     1
15       A50     65                P     1
16       A25     96                P     1
17       A15     85                P     2
18       A33     70                P     1
19       A45     92                P     1
20       A39     93              C++     1
21       A14     70              C++     2
22       A05     71                P     1
23       A2

 40%|█████████████████████████████████▏                                                 | 4/10 [00:00<00:00,  6.52it/s]

...

  Team Name Rating Language of Code Score
1       A45    102                P     4
2       A17     96                P     4
  Team Name Rating Language of Code Score
1       A35     66              C++     3
2       A28     92              C++     2
  Team Name Rating Language of Code Score
1       A22    102              C++     5
2       A08     83              C++     4
  Team Name Rating Language of Code Score
1       A19     72                P     3
2       A04     70                P     1
  Team Name Rating Language of Code Score
1       A42     89                P     4
2       A12     90                P     4
  Team Name Rating Language of Code Score
1       A36     79                P     4
2       A21     79                P     4
  Team Name Rating Language of Code Score
1       A10     63                P     1
2       A34     62                P     3
  Team Name Rating Language of Code Score
1       A15     89                P     5
2       A24     91                

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:01<00:00,  6.02it/s]

   Team Name Rating Language of Code Score
1        A03     90                P     5
2        A09     79              C++     3
3        A14     84              C++     7
4        A27     79                P     5
5        A24     98                P     4
6        A37     75                P     3
7        A11     57                P     3
8        A43    106              C++     7
9        A13     78                P     5
10       A46     65                P     4
11       A07     74                P     5
12       A36     79                P     5
13       A26    110                P     7
14       A41     92                P     4
15       A50     83                P     7
16       A23     96                P     5
17       A10     70                P     3
18       A16     84                P     2
19       A12     87                P     5
20       A49    101                P     6
21       A15     89                P     6
22       A20     94              C++     5
23       A2

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.10it/s]

   Team Name Rating Language of Code Score
1        A26    112                P     8
2        A43    108              C++     8
3        A39    113              C++     8
4        A50     88                P     8
5        A14     89              C++     8
6        A45    108                P     7
7        A49    103                P     7
8        A22    101              C++     7
9        A15     91                P     7
10       A25    105                P     6
11       A48     99                P     6
12       A20     96              C++     6
13       A12     92                P     6
14       A13     83                P     6
15       A07     76                P     6
16       A23     94                P     5
17       A03     88                P     5
18       A42     84                P     5
19       A27     74                P     5
20       A21     77                P     5
21       A18     81                P     5
22       A36     74                P     5
23       A0